# Slider Widget for Multi-Extension FITS Image

This demo shows you how to implement a simple Jupyter widget, which allows you to see a different extension in a given multi-extension FITS (MEF) image via a slider.

Dependencies:

* astropy
* ginga
* ipywidgets
* IPython
* jupyter
* notebook

If widget does not show up, you also need to enable widget in your Jupyter notebooks::

    jupyter nbextension enable --py widgetsnbextension

In [1]:
import ipywidgets as widgets

# Create a Jupyter image that will be our display surface.
# Format can be 'jpeg' or 'png'; specify width and height to set viewer size.
# PNG will be a little clearer, especially with overlaid graphics, but
# JPEG is faster to update.
jup_img = widgets.Image(format='jpeg', width=500, height=500)

In [2]:
from ginga.misc.log import get_logger

# Here, we just get a null logger.
# See other example(s) on how to log to a file.
logger = get_logger("my viewer", null=True)

In [3]:
from ginga.web.jupyterw.ImageViewJpw import EnhancedCanvasView

# Ginga viewer for Jupyter notebook.
v1 = EnhancedCanvasView(logger=logger)

# Set our linkage between the Jupyter widget at Ginga viewer.
v1.set_widget(jup_img)

# Enable all possible keyboard and pointer operations.
bd = v1.get_bindings()
bd.enable_all(True)

In [4]:
%%javascript
/* some magic to keep the cell contents from scrolling
   (when we embed the viewer)
 */
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [5]:
from astropy.io import fits

# Replace this file with a real MEF image you can access.
filename = 'jd2j07f12_flt.fits'

pf = fits.open(filename)

# Display extensions info for sanity check.
pf.info()

Filename: jd2j07f12_flt.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     244   ()      
  1  SCI           1 ImageHDU        95   (2048, 2048)   float32   
  2  ERR           1 ImageHDU        48   (2048, 2048)   float32   
  3  DQ            1 ImageHDU        40   (2048, 2048)   int16   


In [6]:
# Define a function for the slider.
def change_ext(ext):
    """
    Change displayed extension when it slides.
    
    Parameters
    ----------
    ext : int
        Integer value of the extension.
        
    """
    v1.load_hdu(pf[ext])
    v1.show()

In [7]:
from IPython.display import display
from ipywidgets import interact

# Embed the viewer here.
display(jup_img)

# Initialize and display the slider right below the viewer.
interact(change_ext, ext=widgets.IntSlider(min=0,max=len(pf)-1,step=1,value=1));

A Jupyter Widget

A Jupyter Widget

Now, you should see an image displayed in the viewer. When you move the slider, viewer should update to display the selected extension. If you select a primary header (`ext=0`), viewer will give a warning, which you can ignore.

See other example(s) on how you can further interact with and customize your Ginga viewer in Jupyter notebook.

In [8]:
# Close the FITS file handler when you are done.
pf.close()